### 导入数据库

In [135]:
db_path ='/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/self_construct/US_National_js_feb.sqlite'
#db_path ='/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/self_construct/temoa_utopia_del.sqlite'

### 执行sql文件地址

In [136]:

sql_file = '/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/self_construct/US_National_js_feb.sql'
#sql_file = '/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/self_construct/temoa_utopia_del.sql'

In [137]:
# db_path ='/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/self_construct/for_reference/US_Regional.sqlite'
# sql_file = '/Users/jerrysong/Study_Grad/Nankai_Research_with_Python/Nankai_Research/Temoa_2.0_collection/data_files/Temoa_Input_US_based_1.0/US_Regional.sql'

In [138]:
import sqlite3
import os
os.remove(db_path) #清除文件
con = sqlite3.connect(db_path)  # 新建sqlite3数据文件
cs = con.cursor()
print ("数据库清空并打开")

数据库清空并打开


In [139]:
from enum import Enum
# 定义状态的枚举
class State(Enum):
 
    CODE = 0                        # 代码
    SLASH = 1                       # 斜杠
    NOTE_MULTILINE = 2              # 多行注释
    NOTE_MULTILINE_STAR = 3         # 多行注释遇到*
    NOTE_SINGLELINE = 4             # 单行注释
    BACKSLASH = 5                   # 拆行注释
    CODE_CHAR = 6                   # 字符
    CHAR_ESCAPE_SEQUENCE = 7        # 字符中的转义字符
    CODE_STRING = 8                 # 字符串
    STRING_ESCAPE_SEQUENCE = 9      # 字符串中的转义字符
    STRIGULA = 10                   # 短横线
 
# 删除sql语句中的注释
def delete_sql_note(str):
    # TODO 两种情况："--"与"/*...*/"
    s = ""
    # 初始状态定义为代码
    state = State.CODE
    for c in str:
        if state == State.CODE:
            # 遇到单斜杠
            if c == '/':
                # 将状态改为单斜杠
                state = State.SLASH
            # 遇上短横线
            elif c == "-":
                state = State.STRIGULA
            else:
                s += c
                # 在代码中遇到字符
                if c == '\'':
                    state = State.CODE_CHAR
                # 在代码中遇到字符串
                elif c == '\"':
                    state = State.CODE_STRING
 
        # 如果遇到单横线
        elif state == State.STRIGULA:
            # 后面接着跟一个单横线，则说明是单行注释
            if c == '-':
                state = State.NOTE_SINGLELINE
            # 如果后面跟的是其他字符，则将状态转为代码
            else:
                s += "-" + c
                state = State.CODE
 
        # 遇到单斜杠
        elif state == State.SLASH:
            # 单斜杠后面再遇到*号，则说明是多行注释
            if c == '*':
                state = State.NOTE_MULTILINE
            # 单斜杠后面再遇到单斜杠，说明是单行注释
            elif c == '/':
                state = State.NOTE_SINGLELINE
            # 如果是其他，则将其添加到字符串中
            else:
                s += "/"
                s += c
                state = State.CODE
 
        # 遇到多行注释
        elif state == State.NOTE_MULTILINE:
            # 多行注释后面遇到*号
            if c == '*':
                state = State.NOTE_MULTILINE_STAR
            # 多行注释后面换行了
            else:
                if c == '\n':
                    s += '\r\n'
                # 则当前状态还是多行注释
                state = State.NOTE_MULTILINE
        # 多行注释后面又遇到*号
        elif state == State.NOTE_MULTILINE_STAR:
            # 如果*号后面又是单斜杠在，则说明注释结束了
            if c == '/':
                # 将状态改为代码
                state = State.CODE
            # 如果*号后面又是*号，则说明还是多行注释遇到*
            elif c == '*':
                state = State.NOTE_MULTILINE_STAR
            # 其他情况还是多行注释
            else:
                state = State.NOTE_MULTILINE
        # 单行注释
        elif state == State.NOTE_SINGLELINE:
            if c == '\\':
                state = State.BACKSLASH
            # 如果遇到换行符，则说明单行注释结束了
            elif c == '\n':
                s += '\r\n'
                state = State.CODE
            # 如果是其他情况，则说明单行注释还没有结束
            else:
                state = State.NOTE_SINGLELINE
        # 拆行注释
        elif state == State.BACKSLASH:
            if c == '\\' or c == '\r' or c == '\n':
                if c == '\n':
                    s += '\r\n'
                state = State.BACKSLASH
            else:
                state = State.NOTE_SINGLELINE
        # 代码中遇到单引号
        elif state == State.CODE_CHAR:
            s += c
            # 字符串中的转义字符
            if c == '\\':
                state = State.CHAR_ESCAPE_SEQUENCE
            # 再次遇到单引号，则说明字符串内容结束了。
            elif c == '\'':
                state = State.CODE
            # 如果是其他情况，则还是在字符串中
            else:
                state = State.CODE_CHAR
        # 字符串中遇到转义字符
        elif state == State.CHAR_ESCAPE_SEQUENCE:
            s += c
            state = State.CODE_CHAR
        # 字符串
        elif state == State.CODE_STRING:
            s += c
            if c == '\\':
                state = State.STRING_ESCAPE_SEQUENCE
            # 字符串内容结束
            elif c == '\"':
                state = State.CODE
            else:
                state = State.CODE_STRING
        elif state == State.STRING_ESCAPE_SEQUENCE:
            s += c
            state = State.CODE_STRING
    return s

print('导入数据清理函数')   


导入数据清理函数


In [140]:
with open(sql_file,encoding='utf-8',mode='r') as f:
    # 读取整个sql文件，以分号切割。[:-1]删除最后一个元素，也就是空字符串
    f = f.read()
    res = delete_sql_note(f)
    sql_list = res.split(';')[:-1]
    a=0
    for x in sql_list:
        # 判断包含空行的
        if '\n' in x:
            # 替换空行为1个空格
            x = x.replace('\n', ' ')

        # sql语句添加分号结尾
        sql_item = x+';'
        cs.execute(sql_item)
        #print(x)
        a=a+1

con.commit()
cs.close()
con.close()

# print("数据导入成功")